## Set up

In [1]:
import requests 
import pandas as pd
import yfinance as yf
import numpy as np

c:\Users\alver\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\alver\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
API_KEY = 'OGA9PANJZTKOUAW1'
# API_KEY = '2M1YJDW3Y4UWN3R7'

dji_tickers = [
    "AMZN", "AXP", "AMGN", "AAPL", "BA", "CAT", "CSCO", "CVX", "GS", "HD",
    "HON", "IBM", "INTC", "JNJ", "KO", "JPM", "MCD", "MMM", "MRK", "MSFT",
    "NKE", "PG", "TRV", "UNH", "CRM", "VZ", "V", "WMT", "DIS", "DOW"
]

#basket of stocks by industry to get industry averages
finance_symbols = ['HSBC', 'BAC', 'C', 'WFC', 'DB']
industrial_symbols = ['MMM', 'ITW', 'EMR', 'PH', 'TEX']
consumer_symbols = ['WMT', 'EBAY', 'JD', 'BABA', 'ETSY']
tech_symbols = ['ORCL', 'SAP', 'ADBE', 'TEAM', 'MSFT']

our_stocks = ['CRM', 'AMZN', 'JPM', 'GS', 'CAT']
test = ['CRM']

## Sorting the stocks in the DJI

In [3]:
stock_dict = {}
for i in dji_tickers:
	stock_data = yf.Ticker(i)
	hist_stock = stock_data.history(start = pd.to_datetime('2010-01-01'), end = pd.to_datetime('2016-01-01'), interval = '1d')
	sector = stock_data.info.get('sector', 'N/A')
	bt_data = pd.DataFrame()
	bt_data["Close_Price"] = hist_stock["Close"]
	bt_data["Return"] = np.log(bt_data["Close_Price"] / bt_data["Close_Price"].shift(1))
	bt_data = bt_data.dropna()
	if len(bt_data['Close_Price']) != 0:
		stock_dict[i] = {}
		stock_dict[i]['data'] = bt_data
		stock_dict[i]['sector'] = sector

# check standard deviation
for i in stock_dict:
	stock_dict[i]['volatility'] = stock_dict[i]['data']['Return'].std()

sorted_stock_dict = dict(sorted(stock_dict.items(), key=lambda x: x[1]["volatility"], reverse=True))
for stock in sorted_stock_dict:
	print(stock)
	print(sorted_stock_dict[stock]['sector'])
	print(sorted_stock_dict[stock]['volatility'])
	print('-------------------------------------')

$DOW: possibly delisted; no price data found  (1d 2010-01-01 00:00:00 -> 2016-01-01 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1262322000, endDate = 1451624400")


CRM
Technology
0.023509137788098457
-------------------------------------
AMZN
Consumer Cyclical
0.02059007655175249
-------------------------------------
JPM
Financial Services
0.017399476913485822
-------------------------------------
GS
Financial Services
0.017257035558607588
-------------------------------------
CAT
Industrials
0.01717676572138756
-------------------------------------
CSCO
Technology
0.01699416009315366
-------------------------------------
AAPL
Technology
0.01680806337114829
-------------------------------------
V
Financial Services
0.0160295384279765
-------------------------------------
AXP
Financial Services
0.015268462309620161
-------------------------------------
UNH
Healthcare
0.015178125049240036
-------------------------------------
BA
Industrials
0.015103209971189076
-------------------------------------
AMGN
Healthcare
0.015009135256511092
-------------------------------------
INTC
Technology
0.014847112252182394
-------------------------------------
NK

## Method to get EPS and Industry EPS

In [ ]:
def get_eps(symbol):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'EARNINGS',
        'symbol': symbol,
        'apikey': API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    print(data)
    
    eps = []
    
    # Extract EPS data for 2016
    if 'quarterlyEarnings' in data:
        for earnings in data['annualEarnings']:
            fiscal_date = earnings['fiscalDateEnding']
            if fiscal_date.startswith('2016'):
                eps_value = earnings.get('reportedEPS', None)  # Check for reportedEPS
                if eps_value is not None:
                    eps.append(float(eps_value))

    return eps

def industry_eps(industry_name):
    all_eps = []

    for symbol in industry_name:
        eps_values = get_eps(symbol)
        all_eps.extend(eps_values)

    # Calculate average EPS
    if all_eps:
        average_eps = sum(all_eps) / len(all_eps)
        print(industry_name)
        print(all_eps)
        print(f'Average EPS for sector in 2016: {average_eps:.2f}')
    else:
        print('No EPS data found for the specified companies in 2016.')

## Industry EPS

In [4]:
industry_eps(finance_symbols)

{'symbol': 'HSBC', 'annualEarnings': [{'fiscalDateEnding': '2024-09-30', 'reportedEPS': '6.1042'}, {'fiscalDateEnding': '2023-12-31', 'reportedEPS': '2.7903'}, {'fiscalDateEnding': '2022-12-31', 'reportedEPS': '5.07'}, {'fiscalDateEnding': '2021-12-31', 'reportedEPS': '3.35'}, {'fiscalDateEnding': '2020-12-31', 'reportedEPS': '1.2393'}, {'fiscalDateEnding': '2019-12-31', 'reportedEPS': '2.9058'}, {'fiscalDateEnding': '2018-12-31', 'reportedEPS': '3.1'}, {'fiscalDateEnding': '2017-12-31', 'reportedEPS': '2.525'}, {'fiscalDateEnding': '2016-12-31', 'reportedEPS': '0.2975'}, {'fiscalDateEnding': '2015-12-31', 'reportedEPS': '0.5176'}, {'fiscalDateEnding': '2014-12-31', 'reportedEPS': '0.6586'}, {'fiscalDateEnding': '2013-12-31', 'reportedEPS': '0.7673'}, {'fiscalDateEnding': '2012-12-31', 'reportedEPS': '0.9513'}, {'fiscalDateEnding': '2011-12-31', 'reportedEPS': '0.7917'}, {'fiscalDateEnding': '2010-12-31', 'reportedEPS': '0.3885'}], 'quarterlyEarnings': [{'fiscalDateEnding': '2024-09-30

In [23]:
industry_eps(industrial_symbols)

{'symbol': 'MMM', 'annualEarnings': [{'fiscalDateEnding': '2024-09-30', 'reportedEPS': '6.3'}, {'fiscalDateEnding': '2023-12-31', 'reportedEPS': '9.24'}, {'fiscalDateEnding': '2022-12-31', 'reportedEPS': '10.1'}, {'fiscalDateEnding': '2021-12-31', 'reportedEPS': '10.12'}, {'fiscalDateEnding': '2020-12-31', 'reportedEPS': '8.75'}, {'fiscalDateEnding': '2019-12-31', 'reportedEPS': '9.16'}, {'fiscalDateEnding': '2018-12-31', 'reportedEPS': '9.98'}, {'fiscalDateEnding': '2017-12-31', 'reportedEPS': '8.84'}, {'fiscalDateEnding': '2016-12-31', 'reportedEPS': '8.16'}, {'fiscalDateEnding': '2015-12-31', 'reportedEPS': '7.68'}, {'fiscalDateEnding': '2014-12-31', 'reportedEPS': '7.49'}, {'fiscalDateEnding': '2013-12-31', 'reportedEPS': '6.72'}, {'fiscalDateEnding': '2012-12-31', 'reportedEPS': '6.35'}, {'fiscalDateEnding': '2011-12-31', 'reportedEPS': '5.96'}, {'fiscalDateEnding': '2010-12-31', 'reportedEPS': '5.75'}, {'fiscalDateEnding': '2009-12-31', 'reportedEPS': '4.68'}, {'fiscalDateEnding'

In [24]:
industry_eps(consumer_symbols)

{'symbol': 'WMT', 'annualEarnings': [{'fiscalDateEnding': '2024-10-31', 'reportedEPS': '1.27'}, {'fiscalDateEnding': '2024-01-31', 'reportedEPS': '5.44'}, {'fiscalDateEnding': '2023-01-31', 'reportedEPS': '6.28'}, {'fiscalDateEnding': '2022-01-31', 'reportedEPS': '6.45'}, {'fiscalDateEnding': '2021-01-31', 'reportedEPS': '5.54'}, {'fiscalDateEnding': '2020-01-31', 'reportedEPS': '4.94'}, {'fiscalDateEnding': '2019-01-31', 'reportedEPS': '4.92'}, {'fiscalDateEnding': '2018-01-31', 'reportedEPS': '4.41'}, {'fiscalDateEnding': '2017-01-31', 'reportedEPS': '4.33'}, {'fiscalDateEnding': '2016-01-31', 'reportedEPS': '4.59'}, {'fiscalDateEnding': '2015-01-31', 'reportedEPS': '5.07'}, {'fiscalDateEnding': '2014-01-31', 'reportedEPS': '5.13'}, {'fiscalDateEnding': '2013-01-31', 'reportedEPS': '5.02'}, {'fiscalDateEnding': '2012-01-31', 'reportedEPS': '4.48'}, {'fiscalDateEnding': '2011-01-31', 'reportedEPS': '4.09'}, {'fiscalDateEnding': '2010-01-31', 'reportedEPS': '3.66'}, {'fiscalDateEnding'

In [19]:
industry_eps(tech_symbols)

{'symbol': 'ORCL', 'annualEarnings': [{'fiscalDateEnding': '2024-08-31', 'reportedEPS': '1.39'}, {'fiscalDateEnding': '2024-05-31', 'reportedEPS': '5.57'}, {'fiscalDateEnding': '2023-05-31', 'reportedEPS': '5.13'}, {'fiscalDateEnding': '2022-05-31', 'reportedEPS': '4.91'}, {'fiscalDateEnding': '2021-05-31', 'reportedEPS': '4.69'}, {'fiscalDateEnding': '2020-05-31', 'reportedEPS': '3.88'}, {'fiscalDateEnding': '2019-05-31', 'reportedEPS': '3.54'}, {'fiscalDateEnding': '2018-05-31', 'reportedEPS': '3.14'}, {'fiscalDateEnding': '2017-05-31', 'reportedEPS': '2.74'}, {'fiscalDateEnding': '2016-05-31', 'reportedEPS': '2.61'}, {'fiscalDateEnding': '2015-05-31', 'reportedEPS': '2.77'}, {'fiscalDateEnding': '2014-05-31', 'reportedEPS': '2.88'}, {'fiscalDateEnding': '2013-05-31', 'reportedEPS': '2.69'}, {'fiscalDateEnding': '2012-05-31', 'reportedEPS': '2.46'}, {'fiscalDateEnding': '2011-05-31', 'reportedEPS': '2.22'}, {'fiscalDateEnding': '2010-05-31', 'reportedEPS': '1.67'}, {'fiscalDateEnding

## Get earnings growth 

In [5]:
def get_earnings_growth(symbol):
    API_KEY = 'YOUR_API_KEY'  # Replace with your actual API key
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'EARNINGS',
        'symbol': symbol,
        'apikey': API_KEY
    }
    
    response = requests.get(url, params=params)
    data = response.json()

    # Extract earnings data
    earnings = {}
    if 'annualEarnings' in data:
        for entry in data['annualEarnings']:
            year = entry['fiscalDateEnding'][:4]
            eps = float(entry['reportedEPS'])
            if year in ['2010', '2011', '2012', '2013', '2014', '2015', '2016']:
                earnings[year] = eps

    return earnings

def get_average_yoy_growth(earnings_dict):
    # Extract the years and earnings in sorted order
    years = sorted(earnings_dict.keys())
    yoy_changes = []

    for i in range(1, len(years)):
        prev_year = years[i - 1]
        curr_year = years[i]
        
        prev_eps = earnings_dict[prev_year]
        curr_eps = earnings_dict[curr_year]
        
        # Calculate the YoY percentage change if the previous EPS is not zero
        if prev_eps != 0:
            yoy_change = ((curr_eps - prev_eps) / abs(prev_eps)) * 100
            yoy_changes.append(yoy_change)

    # Calculate the average YoY growth
    if yoy_changes:
        average_yoy_growth = sum(yoy_changes) / len(yoy_changes)
        return average_yoy_growth
    else:
        return None  # Return None if no valid YoY changes exist

for stock in our_stocks:
    print(stock)
    earnings = get_earnings_growth(stock)
    avg_yoy_growth = get_average_yoy_growth(earnings)
    print(earnings)
    print(avg_yoy_growth)
    print('')

CRM
{'2016': 0.75, '2015': 0.52, '2014': 0.35, '2013': 0.41, '2012': 0.35, '2011': 0.31, '2010': 0.16}
33.660689068340524

AMZN
{'2016': 0.25, '2015': 0.06, '2014': -0.03, '2013': -0.07, '2012': 0.02, '2011': 0.07, '2010': 0.13}
17.704517704517723

JPM
{'2016': 6.06, '2015': 5.99, '2014': 5.29, '2013': 6.01, '2012': 5.27, '2011': 4.47, '2010': 3.6}
9.754433229482945

GS
{'2016': 16.36, '2015': 18.27, '2014': 17.07, '2013': 15.47, '2012': 14.15, '2011': 4.41, '2010': 13.14}
28.445020440355194

CAT
{'2016': 3.44, '2015': 4.48, '2014': 6.37, '2013': 5.79, '2012': 8.63, '2011': 7.39, '2010': 4.04}
3.9874034419575017



## Method to get Debt-to-Equity Ratio and Industry Debt-to-Equity Ratio

In [8]:
def get_financial_statements(symbol):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'BALANCE_SHEET',
        'symbol': symbol,
        'apikey': API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    print(data)
    return data.get('annualReports', [])

def get_debt_to_equity_2016(symbol):
    financials = get_financial_statements(symbol)
    for report in financials:
        if 'fiscalDateEnding' in report and '2016' in report['fiscalDateEnding']:
            print(report)
            #total_debt = float(report.get('currentDebt', 0))
            #total_shareholder_equity = float(report.get('shortTermDebt', 0)) + float(report.get('longTermDebt', 0))
            total_debt = float(report.get('totalLiabilities', 0))
            total_shareholder_equity = float(report.get('totalShareholderEquity', 0))
            if total_shareholder_equity > 0:  # Avoid division by zero
                return total_debt / total_shareholder_equity
    return None  # If 2016 data is not found or cannot calculate

# Example use for multiple symbols
def industry_debt_to_equity(industry_symbols):
    all_ratios = []
    for symbol in industry_symbols:
        ratio = get_debt_to_equity_2016(symbol)
        if ratio is not None:
            all_ratios.append(ratio)

    if all_ratios:
        average_ratio = sum(all_ratios) / len(all_ratios)
        print(f'Average Debt-to-Equity Ratio for 2016: {average_ratio:.2f}')
    else:
        print('No 2016 Debt-to-Equity data found for the specified companies.')

## Debt to equity for stocks

In [16]:
for stock in our_stocks:
    print(stock)
    print(get_debt_to_equity_2016(stock))
    print('')

CRM
{'symbol': 'CRM', 'annualReports': [{'fiscalDateEnding': '2024-01-31', 'reportedCurrency': 'USD', 'totalAssets': '99823000000', 'totalCurrentAssets': '29074000000', 'cashAndCashEquivalentsAtCarryingValue': '8472000000', 'cashAndShortTermInvestments': '14274000000', 'inventory': 'None', 'currentNetReceivables': '11414000000', 'totalNonCurrentAssets': '68114000000', 'propertyPlantEquipment': '2635000000', 'accumulatedDepreciationAmortizationPPE': 'None', 'intangibleAssets': '53898000000', 'intangibleAssetsExcludingGoodwill': '5278000000', 'goodwill': '48620000000', 'investments': '10570000000', 'longTermInvestments': '4848000000', 'shortTermInvestments': '5802000000', 'otherCurrentAssets': '1561000000', 'otherNonCurrentAssets': '3433000000', 'totalLiabilities': '40177000000', 'totalCurrentLiabilities': '26631000000', 'currentAccountsPayable': 'None', 'deferredRevenue': '19003000000', 'currentDebt': '1371000000', 'shortTermDebt': '1889000000', 'totalNonCurrentLiabilities': '1414800000

## Industry Debt-to-Equity Ratio

In [9]:
industry_debt_to_equity(finance_symbols)

{'symbol': 'HSBC', 'annualReports': [{'fiscalDateEnding': '2023-12-31', 'reportedCurrency': 'USD', 'totalAssets': '3038677000000', 'totalCurrentAssets': '305911000000', 'cashAndCashEquivalentsAtCarryingValue': '305911000000', 'cashAndShortTermInvestments': '305911000000', 'inventory': 'None', 'currentNetReceivables': 'None', 'totalNonCurrentAssets': '1126247000000', 'propertyPlantEquipment': 'None', 'accumulatedDepreciationAmortizationPPE': 'None', 'intangibleAssets': '8164000000', 'intangibleAssetsExcludingGoodwill': '8164000000', 'goodwill': '4323000000', 'investments': 'None', 'longTermInvestments': '1100826000000', 'shortTermInvestments': 'None', 'otherCurrentAssets': '793709000000', 'otherNonCurrentAssets': 'None', 'totalLiabilities': '235159000000', 'totalCurrentLiabilities': '554911000000', 'currentAccountsPayable': 'None', 'deferredRevenue': 'None', 'currentDebt': 'None', 'shortTermDebt': '310135000000', 'totalNonCurrentLiabilities': '235159000000', 'capitalLeaseObligations': '

In [10]:
industry_debt_to_equity(industrial_symbols)

{'symbol': 'MMM', 'annualReports': [{'fiscalDateEnding': '2023-12-31', 'reportedCurrency': 'USD', 'totalAssets': '50580000000', 'totalCurrentAssets': '16379000000', 'cashAndCashEquivalentsAtCarryingValue': '5933000000', 'cashAndShortTermInvestments': '5933000000', 'inventory': '3944000000', 'currentNetReceivables': '4750000000', 'totalNonCurrentAssets': '40446000000', 'propertyPlantEquipment': '7690000000', 'accumulatedDepreciationAmortizationPPE': '15804000000', 'intangibleAssets': '17153000000', 'intangibleAssetsExcludingGoodwill': '1323000000', 'goodwill': '6382000000', 'investments': '220000000', 'longTermInvestments': '74000000', 'shortTermInvestments': '53000000', 'otherCurrentAssets': '326000000', 'otherNonCurrentAssets': '6806000000', 'totalLiabilities': '45712000000', 'totalCurrentLiabilities': '15297000000', 'currentAccountsPayable': '2776000000', 'deferredRevenue': '572000000', 'currentDebt': '2947000000', 'shortTermDebt': '1795000000', 'totalNonCurrentLiabilities': '3187500

In [11]:
industry_debt_to_equity(consumer_symbols)

{'symbol': 'WMT', 'annualReports': [{'fiscalDateEnding': '2024-01-31', 'reportedCurrency': 'USD', 'totalAssets': '252399000000', 'totalCurrentAssets': '76877000000', 'cashAndCashEquivalentsAtCarryingValue': '9867000000', 'cashAndShortTermInvestments': '18316000000', 'inventory': '54892000000', 'currentNetReceivables': '8796000000', 'totalNonCurrentAssets': '179622000000', 'propertyPlantEquipment': '110810000000', 'accumulatedDepreciationAmortizationPPE': '109049000000', 'intangibleAssets': '32213000000', 'intangibleAssetsExcludingGoodwill': '4100000000', 'goodwill': '28113000000', 'investments': '1200000000', 'longTermInvestments': 'None', 'shortTermInvestments': '8449000000', 'otherCurrentAssets': '3322000000', 'otherNonCurrentAssets': '17071000000', 'totalLiabilities': '168538000000', 'totalCurrentLiabilities': '92415000000', 'currentAccountsPayable': '56812000000', 'deferredRevenue': 'None', 'currentDebt': '5050000000', 'shortTermDebt': '878000000', 'totalNonCurrentLiabilities': '54

In [5]:
industry_debt_to_equity(tech_symbols)

{'symbol': 'ORCL', 'annualReports': [{'fiscalDateEnding': '2024-05-31', 'reportedCurrency': 'USD', 'totalAssets': '140976000000', 'totalCurrentAssets': '22554000000', 'cashAndCashEquivalentsAtCarryingValue': '10454000000', 'cashAndShortTermInvestments': '10661000000', 'inventory': '334000000', 'currentNetReceivables': '7874000000', 'totalNonCurrentAssets': '118422000000', 'propertyPlantEquipment': '21536000000', 'accumulatedDepreciationAmortizationPPE': '13282000000', 'intangibleAssets': '69120000000', 'intangibleAssetsExcludingGoodwill': '6890000000', 'goodwill': '62230000000', 'investments': '2930000000', 'longTermInvestments': 'None', 'shortTermInvestments': '207000000', 'otherCurrentAssets': '2864000000', 'otherNonCurrentAssets': '15493000000', 'totalLiabilities': '132272000000', 'totalCurrentLiabilities': '31544000000', 'currentAccountsPayable': '2357000000', 'deferredRevenue': '21092000000', 'currentDebt': '10605000000', 'shortTermDebt': '11905000000', 'totalNonCurrentLiabilities